In [2]:
!pip install wget
!pip install brunel

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=085f86e16f3f5e54e71adec3a9176d7be2ddb773048bd712f01c217d650dddde
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
     |████████████████████████████████| 2.2 MB 739 kB/s eta 0:00:01
     |████████████████████████████████| 198 kB 4.6 MB/s eta 0:00:01
  Created wheel for brunel: filename=brunel-2.6.2-py3-none-any.whl size=2161246 sha256=175ba26e0300d6f79df1c1aa405aa606d8542003f402facffd7fccd9f5ea9837
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/5b/76/52/7910d0d24e582cc63e46b024289b84cdedf5e1919879ca136e
Successfully built brunel


In [3]:
import brunel
import pandas as pd
import numpy as np
import sklearn.pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, LabelBinarizer, OneHotEncoder
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import f1_score, accuracy_score, roc_curve, roc_auc_score
#from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import json
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [4]:
df_claims = pd.read_csv('/project_data/data_asset/ClaimsLocationsHail.csv')
df_claims.head()

,Customer Id,Hail Inches,Customer Latitude,Customer Longitude,Customer Zipcode,Customer City,Customer State,Customer Country,Customer Since,Damage Claim Amount
0,44182,0.0,40.75,-73.99,10001.0,NewYork,NY,US,2015,110
1,44105,0.0,40.71,-74.00,10002.0,NewYork,NY,US,2014,55
2,44106,0.0,40.73,-73.98,10002.0,NewYork,NY,US,2015,42
3,44110,0.0,40.73,-73.99,10002.0,NewYork,NY,US,2014,36
4,44113,0.0,40.72,-73.98,10002.0,NewYork,NY,US,2014,49


In [5]:
df_claims.dtypes

Customer Id              int64
Hail Inches            float64
Customer Latitude      float64
Customer Longitude     float64
Customer Zipcode       float64
Customer City           object
Customer State          object
Customer Country        object
Customer Since           int64
Damage Claim Amount      int64
dtype: object

In [6]:
df_claims.describe()

,Customer Id,Hail Inches,Customer Latitude,Customer Longitude,Customer Zipcode,Customer Since,Damage Claim Amount
count,61393.000000,61393.000000,61393.000000,61393.000000,61392.000000,61393.000000,61393.000000
mean,34351.664294,0.767140,37.982540,-97.548767,61407.976088,2014.274852,1247.153242
std,20436.615629,0.743312,5.499408,16.922506,28478.366213,2.803194,1746.152002
min,1.000000,0.000000,24.550000,-165.450000,10001.000000,1999.000000,-535.000000
25%,15511.000000,0.000000,33.960000,-115.290000,33180.500000,2014.000000,194.000000
50%,34656.000000,0.500000,38.320000,-94.940000,66210.500000,2015.000000,673.000000
75%,52615.000000,1.250000,41.400000,-82.500000,88011.000000,2016.000000,1201.000000
max,68584.000000,2.000000,71.300000,174.100000,99901.000000,2017.000000,12060.000000


In [7]:
df_claims['HailInches'] = df_claims['Hail Inches']

In [8]:
%brunel data('df_claims') bar x(HailInches) y(#count) color(HailInches) tooltip(#all) :: width=1200, height=350 

<IPython.core.display.Javascript object>

In [9]:
df_claims['CustomerState'] = df_claims['Customer State']
df_claims['DamageClaimAmount'] = df_claims['Damage Claim Amount']

In [10]:
%brunel data('df_claims') x(CustomerState) y(DamageClaimAmount) :: width=1200, height=350 

<IPython.core.display.Javascript object>

In [11]:
df_claims.columns

Index(['Customer Id', 'Hail Inches', 'Customer Latitude', 'Customer Longitude',
       'Customer Zipcode', 'Customer City', 'Customer State',
       'Customer Country', 'Customer Since', 'Damage Claim Amount',
       'HailInches', 'CustomerState', 'DamageClaimAmount'],
      dtype='object')

In [12]:
# Define input data to the model
X = df_claims.drop(['Customer Id', 'Customer City', 'CustomerState', 'Customer State', 'Customer Country', 'Customer Zipcode', 'DamageClaimAmount'], axis=1)

In [13]:
X.shape
X

,Hail Inches,Customer Latitude,Customer Longitude,Customer Since,Damage Claim Amount,HailInches
0,0.00,40.75,-73.99,2015,110,0.00
1,0.00,40.71,-74.00,2014,55,0.00
2,0.00,40.73,-73.98,2015,42,0.00
3,0.00,40.73,-73.99,2014,36,0.00
4,0.00,40.72,-73.98,2014,49,0.00
...,...,...,...,...,...,...
61388,0.00,56.70,-132.94,2015,98,0.00
61389,0.00,56.81,-132.94,2015,102,0.00
61390,0.00,55.34,-131.65,2015,13,0.00
61391,0.25,55.45,-131.80,2016,374,0.25


In [14]:
# Define the target variable 
y = df_claims['Damage Claim Amount']

In [15]:
# split the data to training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)

In [16]:
#import warnings
#warnings.filterwarnings("ignore")

In [17]:
#Instanitate the ExtraTreesRegressor
reg = ExtraTreesRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)

In [18]:
#The ExtraTreesRegressor score
reg.score(X_test, y_test)

0.9999925349750988

In [19]:
### call pipeline.predict() on your X_test data to make a set of test predictions
y_prediction = reg.predict( X_test )

In [20]:
# show first 10 rows of claim amount predictions
y_prediction[0:10,]

array([ 997.  ,  157.  , 1161.  , 5600.  ,  823.  , 2179.99,  460.  ,
        610.  ,  348.  ,   31.  ])

In [21]:
### test your predictions using sklearn.mean_absolute_error()
report = sklearn.metrics.mean_absolute_error( y_test, y_prediction )

### and print the report
print(report)

0.10033174068846037


In [22]:
# from sklearn.model_selection import RepeatedKFold
# from sklearn.model_selection import cross_val_score
# from numpy import mean
# from numpy import std

# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# n_scores = cross_val_score(reg, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# # report performance
# print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

## User Inputs
The user can specify the name for the space, models and model deployments.
Model and its deployment will be stored in the deployment space. The deployment space can be created programmatically using `ibm-watson-machine-learning` client.
If a space with specified space_name already exists, user can either use the existing space by specifying `use_existing_space=True` or delete the existing space and create a new one by specifying `use_existing_space=False` below. By default `use_existing_space` is set to True.

In [23]:
# Specify a names for the space being created, the saved model and the model deployment
space_name = 'Severe Weather Risk Deployment Space'

model_name = 'severe_weather_risk_python_model'
model_tag = 'severe_weather_risk_python_model_tag'
model_desc = "Random Forest"

deployment_name = 'severe_weather_risk_python_model_deployment'
deployment_tag = 'severe_weather_risk_python_model_deployment_tag'

use_existing_space=True

In [24]:

from ibm_watson_machine_learning import APIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.5"
}

client = APIClient(wml_credentials)

### Create the Deployment Space
Create a new deployment space using name of the space as specified in the user inputs cell above. The space name will be used in future to identify this space.

By default, if the space with specified name already exists, the existing space will be used to store and deploy the models.

In [25]:
space_uid=""
for space in client.spaces.get_details()['resources']:

    if space['entity']['name'] ==space_name:
        print("Deployment space with ",space_name,"already exists . .")
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)
        if(use_existing_space==False):

            for deployment in client.deployments.get_details()['resources']:
                print("Deleting deployment",deployment['entity']['name'], "in the space",)
                deployment_id=deployment['metadata']['id']
                client.deployments.delete(deployment_id)
            print("Deleting Space ",space_name,)
            client.spaces.delete(space_uid)
            time.sleep(5)
        else:
            print("Using the existing space")
            
            
if (space_uid=="" or use_existing_space==False):
    print("\nCreating a new deployment space -",space_name)
    # create the space and set it as default
    space_meta_data = {
        client.spaces.ConfigurationMetaNames.NAME : space_name

        }

    stored_space_details = client.spaces.store(space_meta_data)

    space_uid = stored_space_details['metadata']['id']

    client.set.default_space(space_uid)

Deployment space with  Severe Weather Risk Deployment Space already exists . .
Using the existing space


### Store the Pipeline Details
Store the pipeline in the deployment space. Again, the name for the pipeline is specified in the user inputs cell above. Software specification and default tag for the model are specified in the model metadata.
The Software Specification refers to the runtime used in the Notebook, WML training and WML deployment. We use the software specification `default_py3.7` to store the models. We get the ID of the software specification and include it in the metadata when storing the model. Available Software specifications can be retrieved using `client.software_specifications.list()`.

In [30]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
tensorflow_1.15-py3.6          2b73a275-7cbf-420b-a912-eae7f436e0bc  base
pytorch_1.2-py3.6              2c8ef57d-2687-4b7d-acce-01f94976dac1  base
spark-mllib_2.3                2e51f70

In [26]:
sw_spec_uid = client.software_specifications.get_uid_by_name("default_py3.7")

metadata = {
    client.repository.ModelMetaNames.NAME: model_name,
    client.repository.ModelMetaNames.DESCRIPTION: model_desc,
    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.23",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid ,
    client.repository.ModelMetaNames.TAGS: [model_tag],
    client.repository.ModelMetaNames.SPACE_UID: space_uid
}

stored_model_details = client.repository.store_model(model=reg, meta_props=metadata, training_data=X_train, training_target=y_train, feature_names=X_train.columns)
stored_model_details

{'entity': {'label_column': 'Damage Claim Amount',
  'software_spec': {'id': 'e4429883-c883-42b6-87a8-f419d64088cd',
   'name': 'default_py3.7'},
  'training_data_references': [{'connection': {'access_key_id': 'not_applicable',
     'endpoint_url': 'not_applicable',
     'secret_access_key': 'not_applicable'},
    'id': '1',
    'location': {},
    'schema': {'fields': [{'name': 'Hail Inches', 'type': 'float64'},
      {'name': 'Customer Latitude', 'type': 'float64'},
      {'name': 'Customer Longitude', 'type': 'float64'},
      {'name': 'Customer Since', 'type': 'int64'},
      {'name': 'Damage Claim Amount', 'type': 'int64'},
      {'name': 'HailInches', 'type': 'float64'}],
     'id': '1',
     'type': 'DataFrame'},
    'type': 'fs'}],
  'type': 'scikit-learn_0.23'},
 'metadata': {'created_at': '2020-11-10T15:54:52.889Z',
  'description': 'Random Forest',
  'id': '1f1c9e8e-42c6-45ab-93fc-fb60f6cc60f0',
  'modified_at': '2020-11-10T15:55:41.622Z',
  'name': 'severe_weather_risk_pyth

### Deploy the Model Pipeline
Deploy the saved model. The deployment name is specified in the user inputs cell above. A default tag for the deployment is specified in the deployment metadata.


In [27]:
# deploy the model pipeline
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: deployment_name,
    client.deployments.ConfigurationMetaNames.TAGS : [ deployment_tag],
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# deploy the model

model_uid = stored_model_details["metadata"]["id"]
deployment_details = client.deployments.create( artifact_uid=model_uid, meta_props=meta_props)
deployment_details



#######################################################################################

Synchronous deployment creation for uid: '1f1c9e8e-42c6-45ab-93fc-fb60f6cc60f0' started

#######################################################################################


initializing.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2ccc7c6f-d4f3-4f6d-936b-8409cc22ab52'
------------------------------------------------------------------------------------------------




{'entity': {'asset': {'id': '1f1c9e8e-42c6-45ab-93fc-fb60f6cc60f0'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'S', 'num_nodes': 1},
  'name': 'severe_weather_risk_python_model_deployment',
  'online': {},
  'space_id': '110278ea-003d-4deb-9416-315649c0db8c',
  'status': {'online_url': {'url': 'https://internal-nginx-svc:12443/ml/v4/deployments/2ccc7c6f-d4f3-4f6d-936b-8409cc22ab52/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2020-11-10T15:55:43.077Z',
  'id': '2ccc7c6f-d4f3-4f6d-936b-8409cc22ab52',
  'modified_at': '2020-11-10T15:55:43.077Z',
  'name': 'severe_weather_risk_python_model_deployment',
  'owner': '1000331001',
  'space_id': '110278ea-003d-4deb-9416-315649c0db8c',
  'tags': ['severe_weather_risk_python_model_deployment_tag']}}

### Sample Scoring
Now we can test the model by scoring a sample record on the deployment as below.

In [28]:

# Select sample customer ids to score
CUSTOMER_IDS=[249,250,310,311]
# Filter these customers records from df_raw
df_score=df_claims[df_claims['Customer Id'].isin(CUSTOMER_IDS)]

In [29]:
# Create a payload with fields and values dictionary
fields=X_train.columns.tolist()
values=df_score[fields].values.tolist()
payload_scoring={"input_data": [{"fields": fields,"values": values}]}
print("Customer ids",CUSTOMER_IDS,"\n\n Predicted Damage Claim Amount:")
#Get the deployment id for the deployed model
deployment_uid=client.deployments.get_uid(deployment_details)


payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}

# Pass the payload to wml client to predict attrition_status for the sample records
scoring_response = client.deployments.score(deployment_uid, payload_scoring)
scoring_response

Customer ids [249, 250, 310, 311] 

 Predicted Damage Claim Amount:


{'predictions': [{'fields': ['prediction'],
   'values': [[98.0], [102.0], [13.0], [374.0]]}]}